<a href="https://colab.research.google.com/github/bhuvi6/F.M.M.L-LABS-PROJECTS/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [14]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [15]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


Yes, averaging the validation accuracy across multiple splits generally provides more consistent and reliable results. This approach is often used in cross-validation, where the dataset is split into multiple subsets (folds). Here’s why averaging across splits is beneficial:

1. **Reduces Variance**: Individual splits may lead to variance in accuracy due to different training and validation sets. By averaging the accuracy across multiple splits, you reduce the impact of this variance, leading to a more stable and consistent estimate of model performance.

2. **Mitigates Overfitting/Underfitting**: Relying on a single split might cause misleading results if that particular split is not representative of the overall dataset. Multiple splits ensure that the model is tested against different subsets of the data, reducing the likelihood of overfitting or underfitting to a specific subset.

3. **Better Generalization Estimate**: Averaging accuracy across splits provides a better estimate of how well the model will perform on unseen data, as it takes into account different possible training/validation data configurations.

4. **Comprehensive Evaluation**: It allows you to assess model performance in a more comprehensive way, as it ensures that every data point in the dataset has been used for both training and validation across different splits.

Overall, this method provides a more reliable measure of your model's generalization ability.

Averaging validation accuracy across multiple splits typically provides a more accurate estimate of test accuracy compared to using a single train/validation split. Here's why:

1. **Representative Sampling**: Multiple splits ensure that your model is trained and validated on different portions of the data, which reduces the chance that the model's performance is dependent on a particular data configuration. This makes the estimate of test accuracy more representative of how the model will perform on unseen data.

2. **Error Reduction**: A single split might be unrepresentative of the entire dataset, leading to an inaccurate estimate of test accuracy. For example, if the validation set in a single split is either too easy or too hard, it could skew the accuracy. By averaging over multiple splits, you reduce the impact of these outliers.

3. **Lower Variance**: Averaging across multiple validation splits reduces variance in the accuracy estimate. Since the test accuracy is a measure of the model's ability to generalize, lower variance in the validation accuracy suggests that the model's performance is stable, which generally leads to a more accurate estimate of test accuracy.

4. **Cross-Validation**: Techniques like k-fold cross-validation, where the data is split into k subsets and the model is trained and validated k times, each time with a different validation set, provide a robust estimate of test accuracy. The average accuracy from cross-validation is often used as a reliable predictor of test performance.

In summary, averaging validation accuracy across multiple splits typically gives a more accurate and reliable estimate of test accuracy because it accounts for variability in the data and reduces the likelihood of overfitting to any particular subset.

Increasing the number of iterations (or splits) generally leads to a better estimate of test accuracy, but there are diminishing returns after a certain point. Here’s how the number of iterations affects the estimate:

### 1. **Improved Stability and Reliability**:
   - **Higher Iterations**: With more iterations, the estimate becomes more stable because it averages out more of the variability in the data. This leads to a more reliable estimate of test accuracy, as the model is tested on a broader range of data configurations.
      - **Lower Iterations**: Fewer iterations might result in an estimate that is more susceptible to the specific characteristics of the small number of splits. This could lead to an estimate that doesn’t generalize well to unseen data.

      ### 2. **Reduced Variance**:
         - **Higher Iterations**: The variance in the accuracy estimate decreases as the number of iterations increases. This means that the confidence in the accuracy estimate improves, making it a better predictor of the model's performance on new data.
            - **Lower Iterations**: Fewer iterations might result in higher variance in the accuracy estimates, meaning the estimate could fluctuate significantly depending on the specific splits used.

            ### 3. **Diminishing Returns**:
               - **Beyond a Certain Point**: While increasing the number of iterations does lead to better estimates initially, the improvement becomes marginal beyond a certain point. For example, moving from 5 to 10 iterations may significantly improve the estimate, but moving from 50 to 100 iterations might not offer much additional benefit.
                  - **Computational Cost**: More iterations also mean higher computational costs, so it's important to balance the need for accurate estimates with the resources available.

                  ### 4. **Bias-Variance Tradeoff**:
                     - **Higher Iterations**: As you increase the number of iterations, you generally reduce the bias in your accuracy estimate because the model is evaluated more thoroughly. However, after a certain point, the variance decreases only slightly, and the bias-variance tradeoff stabilizes.
                        - **Lower Iterations**: With fewer iterations, the estimate might be biased towards the particular splits used, which can give an inaccurate picture of the model’s performance.

                        ### Conclusion:
                        Yes, you do generally get a better estimate with higher iterations because it leads to more stable, reliable, and less variable estimates. However, after a certain point, the benefits of additional iterations diminish, and the computational cost may outweigh the slight gains in accuracy. Therefore, it's often advisable to use a reasonable number of iterations (e.g., 5-10 folds in cross-validation) that balances accuracy with computational efficiency.

Increasing the number of iterations can help mitigate some of the challenges associated with a very small training or validation dataset, but it has its limitations. Here’s how it works and where it might fall short:

### 1. **Better Utilization of Data**:
   - **Higher Iterations**: When you have a very small dataset, increasing the number of iterations (e.g., through techniques like k-fold cross-validation) ensures that the model is trained and validated on as much of the data as possible. This can help in making the most out of the limited data, providing a more comprehensive evaluation of the model’s performance.
      - **Data Scarcity**: However, the fundamental issue of data scarcity remains. Even with more iterations, the model might still not capture all the underlying patterns in the data due to the small sample size.

      ### 2. **Reduced Overfitting**:
         - **Higher Iterations**: For small datasets, models are at a higher risk of overfitting, meaning they perform well on the training data but poorly on unseen data. Increasing the number of iterations can help by reducing the variance in the validation accuracy, making the performance estimate more reliable and reducing the likelihood of overfitting.
            - **Limited Effect**: That said, if the dataset is extremely small, the risk of overfitting remains high, regardless of the number of iterations. The model may still not generalize well to new data.

            ### 3. **Variance Reduction**:
               - **Higher Iterations**: More iterations can help reduce the variance in the performance estimate, leading to a more stable assessment of the model. This is particularly important in small datasets where variance might otherwise be high.
                  - **Insufficient Data**: Despite the reduction in variance, the underlying issue of insufficient data might still lead to biased estimates. The model's performance might still be overly optimistic or pessimistic, depending on the specific samples in the dataset.

                  ### 4. **Bias-Variance Tradeoff**:
                     - **Higher Iterations**: Increasing iterations can help in balancing the bias-variance tradeoff, which is crucial for small datasets. It reduces the impact of any single validation set on the overall estimate.
                        - **Inherent Bias**: However, with a very small dataset, the model might still have high bias because it doesn't have enough information to learn effectively. Even with more iterations, the model may struggle to capture the true underlying distribution.

                        ### 5. **Computational Considerations**:
                           - **Higher Iterations**: While increasing iterations can improve the reliability of performance estimates, it also increases computational costs, which might be a concern if the model is complex and the dataset, though small, requires significant processing.
                              - **Efficiency vs. Accuracy**: You must balance the need for more iterations with the available computational resources, especially since the gains from additional iterations might be marginal in very small datasets.

                              ### Conclusion:
                              While increasing the number of iterations can help make the most of a small dataset by providing more reliable and stable estimates of model performance, it does not completely solve the problems associated with small training or validation datasets. The fundamental issues of data scarcity, such as the risk of overfitting and limited generalization ability, still persist.

                              In practice, if you are dealing with a very small dataset, you might need to consider additional strategies alongside increasing iterations, such as:

                              - **Data Augmentation**: Artificially increasing the size of the dataset by creating variations of the existing data.
                              - **Transfer Learning**: Leveraging a pre-trained model on a similar task to reduce the need for a large training dataset.
                              - **Regularization**: Applying regularization techniques to prevent overfitting.

                              These strategies, in combination with increasing the number of iterations, can help improve model performance when dealing with small datasets.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.